In [1]:
import csv
import json
from kafka import KafkaProducer

from pyspark.sql import SparkSession

In [31]:

# Initialize the Kafka producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)


# Read the CSV file and send each row as a message to Kafka
csv_file_path = 'D:/Python/Kairiz TASK/train_and_test2.csv'

with open(csv_file_path, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        producer.send('real-time-Titanic', value=row)
        print(f'Sent: {row}')

# Flush and close the producer
producer.flush()
producer.close()


Sent: {'Passengerid': '1', 'Age': '22', 'Fare': '7.25', 'Sex': '0', 'sibsp': '1', 'zero': '0', 'Parch': '0', 'Pclass': '3', 'Embarked': '2', '2urvived': '0'}
Sent: {'Passengerid': '2', 'Age': '38', 'Fare': '71.2833', 'Sex': '1', 'sibsp': '1', 'zero': '0', 'Parch': '0', 'Pclass': '1', 'Embarked': '0', '2urvived': '1'}
Sent: {'Passengerid': '3', 'Age': '26', 'Fare': '7.925', 'Sex': '1', 'sibsp': '0', 'zero': '0', 'Parch': '0', 'Pclass': '3', 'Embarked': '2', '2urvived': '1'}
Sent: {'Passengerid': '4', 'Age': '35', 'Fare': '53.1', 'Sex': '1', 'sibsp': '1', 'zero': '0', 'Parch': '0', 'Pclass': '1', 'Embarked': '2', '2urvived': '1'}
Sent: {'Passengerid': '5', 'Age': '35', 'Fare': '8.05', 'Sex': '0', 'sibsp': '0', 'zero': '0', 'Parch': '0', 'Pclass': '3', 'Embarked': '2', '2urvived': '0'}
Sent: {'Passengerid': '6', 'Age': '28', 'Fare': '8.4583', 'Sex': '0', 'sibsp': '0', 'zero': '0', 'Parch': '0', 'Pclass': '3', 'Embarked': '1', '2urvived': '0'}
Sent: {'Passengerid': '7', 'Age': '54', 'Fare'

In [32]:


# spark = SparkSession.builder \
#     .appName("RealTimePipeline") \
#     .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.10:2.1.0") \
#     .getOrCreate()         


In [2]:
from kafka import KafkaConsumer
import pandas as pd
import json

# Kafka broker(s) and topic to consume from
bootstrap_servers = 'localhost:9092'
topic = 'real-time-Titanic'

# Consumer configuration
consumer = KafkaConsumer(topic,
                        bootstrap_servers=bootstrap_servers,
                        group_id='myConsumer',
                        auto_offset_reset='earliest')

# Initialize an empty list to collect data
data_list = []

try:
    # Poll for messages
    for msg in consumer:
        # Decode and parse JSON message
        data = json.loads(msg.value.decode('utf-8'))
        
        # Print received message
        print('Received message:', data)
        
        # Append data to list
        data_list.append(data)
        
        # Check if Passengerid is 1309 and break the loop
        if data.get("Passengerid") == "1309":
            print("Reached Passengerid 1309. Stopping consumption.")
            break

except KeyboardInterrupt:
    print('Aborted by user')

finally:
    # Close consumer
    consumer.close()

# Convert list of dictionaries to DataFrame
OG_DF = pd.DataFrame(data_list)

Received message: {'Passengerid': '1', 'Age': '22', 'Fare': '7.25', 'Sex': '0', 'sibsp': '1', 'zero': '0', 'Parch': '0', 'Pclass': '3', 'Embarked': '2', '2urvived': '0'}
Received message: {'Passengerid': '2', 'Age': '38', 'Fare': '71.2833', 'Sex': '1', 'sibsp': '1', 'zero': '0', 'Parch': '0', 'Pclass': '1', 'Embarked': '0', '2urvived': '1'}
Received message: {'Passengerid': '3', 'Age': '26', 'Fare': '7.925', 'Sex': '1', 'sibsp': '0', 'zero': '0', 'Parch': '0', 'Pclass': '3', 'Embarked': '2', '2urvived': '1'}
Received message: {'Passengerid': '4', 'Age': '35', 'Fare': '53.1', 'Sex': '1', 'sibsp': '1', 'zero': '0', 'Parch': '0', 'Pclass': '1', 'Embarked': '2', '2urvived': '1'}
Received message: {'Passengerid': '5', 'Age': '35', 'Fare': '8.05', 'Sex': '0', 'sibsp': '0', 'zero': '0', 'Parch': '0', 'Pclass': '3', 'Embarked': '2', '2urvived': '0'}
Received message: {'Passengerid': '6', 'Age': '28', 'Fare': '8.4583', 'Sex': '0', 'sibsp': '0', 'zero': '0', 'Parch': '0', 'Pclass': '3', 'Embarke

In [4]:
df = OG_DF 

print("\nDataFrame:")
print(df)


DataFrame:
     Passengerid   Age     Fare Sex sibsp zero Parch Pclass Embarked 2urvived
0              1    22     7.25   0     1    0     0      3        2        0
1              2    38  71.2833   1     1    0     0      1        0        1
2              3    26    7.925   1     0    0     0      3        2        1
3              4    35     53.1   1     1    0     0      1        2        1
4              5    35     8.05   0     0    0     0      3        2        0
...          ...   ...      ...  ..   ...  ...   ...    ...      ...      ...
1304        1305    28     8.05   0     0    0     0      3        2        0
1305        1306    39    108.9   1     0    0     0      1        0        0
1306        1307  38.5     7.25   0     0    0     0      3        2        0
1307        1308    28     8.05   0     0    0     0      3        2        0
1308        1309    28  22.3583   0     1    0     1      3        0        0

[1309 rows x 10 columns]


In [5]:
from Model import Logistic

In [6]:
import pickle

# Load the model
with open('trained_model.pkl', 'rb') as file:
    model = pickle.load(file)
    
    

In [7]:
Cols = df.columns
for i in Cols:
    if df[i].sum() == 0:
        df = df.drop(i,axis=1)
        
df = df.drop( ['Passengerid' ],axis=1)

df  = df.fillna(0)

TestSamplesX = df.iloc[:, :7]
TestSamplesY = df['2urvived']

TestSamplesX

,Age,Fare,Sex,sibsp,zero,Parch,Pclass
0,22,7.25,0,1,0,0,3
1,38,71.2833,1,1,0,0,1
2,26,7.925,1,0,0,0,3
3,35,53.1,1,1,0,0,1
4,35,8.05,0,0,0,0,3
...,...,...,...,...,...,...,...
1304,28,8.05,0,0,0,0,3
1305,39,108.9,1,0,0,0,1
1306,38.5,7.25,0,0,0,0,3
1307,28,8.05,0,0,0,0,3


In [8]:
listTestSamplesY = list(TestSamplesY)

pred = model.predict(TestSamplesX)
# pred = Model.predict(TestSamplesX)

print("Predicted",pred)
listTestSamplesY = list(TestSamplesY)
print("Actual   ",listTestSamplesY)


TypeError: 'float' object cannot be interpreted as an integer